In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import torch.optim as optim
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import torch.nn as nn 
import torch.nn.functional as F 
import matplotlib.pyplot as plt
import operator
from datetime import datetime
import time
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset , TensorDataset
from utilities import *
from model import *
import fds
import os
import logging
import re
import time
from multiprocessing import Pool
import concurrent.futures
from itertools import zip_longest
from scipy.stats import rankdata
import pickle
from typing import Union
from loss import *
from factors import *
from datasets.dataset import *

In [ ]:
dataset1 , df_keys1  = time_series_dataset_dict(
            begin_date = train_begin_date,  end_date = test_end_date,
            prediction_size = 15, adds = False, window = 30, 
            drop = False, norm = True , freq = '60m', ex = True, outstand = False,
            drop0930 = False, ratio = True, winsor = True)
# dataset2 , df_keys2  = time_series_dataset_dict_cross(
#             begin_date = '2018-01-03',  end_date = '2018-03-03',
#             threshold_of_grow_and_decline = 0.5, 
#             prediction_size = 25, adds = False,
#             window = 50, drop = False, norm = True , freq = '30m', ex = False, outstand = False) 

In [4]:
dataset1[pd.Timestamp('2018-01-05')].tensors[0]

tensor([[[ 8.1934e-01,  9.3152e-01,  8.5636e-01,  ..., -4.1639e-01,
          -2.2043e-01, -2.2034e-01],
         [ 8.2998e-01,  5.9495e-01,  7.0799e-01,  ..., -4.2399e-01,
          -2.8815e-01, -2.8840e-01],
         [ 6.1530e-01,  2.3801e-01,  4.0362e-01,  ..., -4.8006e-01,
          -3.1006e-01, -3.0959e-01],
         ...,
         [ 2.0882e+00,  1.3541e-01,  1.6792e+00,  ..., -2.8170e-01,
          -9.9825e-02, -9.9659e-02],
         [ 5.4087e-01, -2.1889e-01,  1.6410e-01,  ..., -1.6877e-01,
          -1.0051e-01, -1.0046e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.7161e-01,
          -1.0355e+00, -1.0353e+00]],

        [[-6.0761e-01, -9.7300e-01, -1.0974e+00,  ..., -5.3252e-01,
          -1.5175e-01, -1.5123e-01],
         [-5.6679e-01, -9.8252e-01, -1.0275e+00,  ..., -7.8841e-01,
          -3.6053e-01, -3.6314e-01],
         [-5.8633e-01, -1.4136e+00, -1.4128e+00,  ...,  7.6923e-02,
           1.0253e+00,  1.0286e+00],
         ...,
         [-2.6252e-01, -2

In [5]:
with open('./datasets/datasets.pkl', 'wb') as f:
    pickle.dump(dataset1, f)
df_keys1.to_pickle('./datasets/datasets_keys.pkl')

In [2]:
train_begin_date = '2015-01-03'
train_end_date = '2020-12-31' 
valid_begin_date = '2021-01-01'
valid_end_date = '2021-03-31'
test_begin_date = '2021-04-01'
test_end_date = '2023-10-31'

In [21]:
train_dataset = {}
valid_dataset = {}
test_dataset = {}
tradingdays = fds.range_trading_days(pd.Timestamp(train_begin_date), pd.Timestamp(valid_end_date), dtype=None)
valdays = select_random_percentage(tradingdays, 0.08)
for timestamp, data in dataset1.items():
    if pd.Timestamp(test_begin_date) <= timestamp <= pd.Timestamp(test_end_date):
        test_dataset[timestamp] = data
    elif timestamp in valdays:
        valid_dataset[timestamp] = data
    elif pd.Timestamp(train_begin_date) <= timestamp < pd.Timestamp(train_end_date):
        train_dataset[timestamp] = data

In [26]:
train_dataset[pd.Timestamp('2020-01-03')].tensors[0]

tensor([[[2.4236e+03, 2.4381e+03, 2.4496e+03,  ..., 8.7128e-01,
          4.5349e-01, 3.8088e-01],
         [2.4525e+03, 2.4063e+03, 2.4612e+03,  ..., 8.7128e-01,
          4.5944e-01, 3.8088e-01],
         [2.4106e+03, 2.4424e+03, 2.4453e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         ...,
         [2.3933e+03, 2.3760e+03, 2.4020e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         [2.4048e+03, 2.4366e+03, 2.4482e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         [2.4467e+03, 2.4814e+03, 2.5002e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01]],

        [[3.1811e+03, 3.2000e+03, 3.2345e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         [3.1989e+03, 3.1894e+03, 3.1989e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         [3.1870e+03, 3.1645e+03, 3.1965e+03,  ..., 8.7128e-01,
          4.5336e-01, 3.8088e-01],
         ...,
         [3.7197e+03, 3.8182e+03, 3.8503e+03,  ..., 8.7128e-01,
          4.533

In [9]:
bars = fds.exbar_mini(fds.shift_trading_day('2018-01-03', -30-1), fds.shift_trading_day('2018-03-03', 10 + 1), freq='60m')

In [12]:
bars['time'].unique()

<TimedeltaArray>
['0 days 09:30:00', '0 days 10:30:00', '0 days 11:30:00', '0 days 14:00:00',
 '0 days 15:00:00']
Length: 5, dtype: timedelta64[ns]

In [31]:
bars = bars[['open','close','high','low','volume','turnover','trade_num', #高开低收量额 ，买卖， 大小单
             'init_buy_trade_num','init_buy_trade_vol','init_buy_trade_val','init_sell_trade_num','init_sell_trade_vol','init_sell_trade_val',
             'large_trade_num','large_trade_vol','large_trade_val','med_trade_num','med_trade_vol','med_trade_val','small_trade_num','small_trade_vol','small_trade_val']]

In [32]:
bars

,open,close,high,low,volume,turnover,trade_num,init_buy_trade_num,init_buy_trade_vol,init_buy_trade_val,...,init_sell_trade_val,large_trade_num,large_trade_vol,large_trade_val,med_trade_num,med_trade_vol,med_trade_val,small_trade_num,small_trade_vol,small_trade_val
0,12.70,12.85,12.86,12.61,61495420.0,7.830772e+08,20196.0,10026.0,32086953.0,4.090911e+08,...,3.739861e+08,674.0,24240327.0,3.086401e+08,3184.0,21769071.0,2.772547e+08,16338.0,15486022.0,1.971824e+08
1,15.00,15.16,15.19,14.60,21647707.0,3.234194e+08,12034.0,6078.0,12059227.0,1.803802e+08,...,1.430392e+08,170.0,4273839.0,6.382030e+07,1671.0,9863103.0,1.474772e+08,10193.0,7510765.0,1.121220e+08
2,5.84,5.86,5.87,5.67,12363913.0,7.100768e+07,6047.0,2906.0,6006365.0,3.457688e+07,...,3.643080e+07,24.0,1250206.0,7.176886e+06,268.0,3680301.0,2.112995e+07,5755.0,7433406.0,4.270084e+07
3,16.69,17.36,17.48,16.40,1720780.0,2.946716e+07,2804.0,1946.0,1126036.0,1.930486e+07,...,1.016230e+07,5.0,193800.0,3.365668e+06,69.0,359800.0,6.119045e+06,2730.0,1167180.0,1.998245e+07
4,5.16,5.21,5.22,5.12,26942026.0,1.392290e+08,13129.0,6160.0,13047027.0,6.752211e+07,...,7.170691e+07,31.0,2340775.0,1.209720e+07,507.0,7021794.0,3.627987e+07,12591.0,17579457.0,9.085196e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,15.45,15.79,15.97,15.20,15131622.0,2.375781e+08,15800.0,8206.0,8074803.0,1.269723e+08,...,1.106058e+08,50.0,1136200.0,1.785632e+07,1078.0,5097541.0,8.006959e+07,14672.0,8897881.0,1.396522e+08
3495,64.17,65.48,67.17,63.04,4366160.0,2.860375e+08,12429.0,6364.0,2243529.0,1.470060e+08,...,1.390315e+08,92.0,471704.0,3.092988e+07,1355.0,1609242.0,1.053716e+08,10982.0,2285214.0,1.497360e+08
3496,40.06,40.72,41.48,39.80,5293344.0,2.159213e+08,11258.0,5832.0,2744786.0,1.119869e+08,...,1.039344e+08,48.0,402645.0,1.641186e+07,1116.0,1963963.0,8.012683e+07,10094.0,2926736.0,1.193826e+08
3497,50.15,50.47,52.40,49.52,9712833.0,4.942958e+08,22183.0,10854.0,4651833.0,2.370227e+08,...,2.572731e+08,190.0,1364926.0,6.933234e+07,2563.0,3735567.0,1.903054e+08,19430.0,4612340.0,2.346580e+08


In [30]:
pd.Timestamp('2018-01-05') in fds.range_trading_days(pd.Timestamp('2018-01-05'), pd.Timestamp('2018-03-05'), dtype=None)

True

In [10]:
for key, tensor in dataset1.items():
    try:
        assert dataset1[key].tensors[0].shape[0] == dataset2[key].tensors[0].shape[0]
    except AssertionError:
        print(f"Dim Assertion failed for key: {key}")

In [27]:
dataframe = pd.read_parquet('../factor_shared/factors_outstanding_0429.parq')
dataframe = dataframe.drop(columns = ['amount_t_ask_hlspread_res_abs_order_99tot','AMOUNT_DEAL_per_depth_ex_kurtosis_PM_cancel_day_ao_25mean_25std','VWAP_depth_analysis_spectrum_mean_PM_order_day_bs_5mean'])

In [71]:
dataframe = dataframe.drop(columns = ['AMOUNT_DEAL_depth_analysis_alpha_ALL_cancel_day_as_abs_3mean'])

In [63]:
dataframe['spread_mean_ab_hlspread_10corr_order_90m'] = -dataframe['spread_mean_ab_hlspread_10corr_order_90m']

In [72]:
dataframe

,amount_deal_ask_hlspread_diff_abs_order_90m_10d_mean,amount_deal_ask_hlspread_diff_abs_cancel_99tot_10d_mean,amount_count_ask_25corr_order_90m,count_ms_ab_hlspread_20corr_order_90m,count_ms_ab_hlspread_20corr_order_99tot,spread_mean_ab_hlspread_10corr_order_90m,spread_mean_ab_hlspread_10corr_match_90m,amount_t_ask_hlspread_20corr_match_90m,amount_t_ask_hlspread_20corr_cancel_90m,vwap_height_at_p2_match_day_bo_3mean,...,amount_d_distance_ratio_12_13_order_m_bs,amount_d_height_at_p1_order_m_bs_3mean,amount_t_triangle_perimeter_order_day_bs_5mean,amount_t_triangle_perimeter_order_m_bo_5mean,amount_t_triangle_perimeter_order_day_as_5mean,amount_t_25_corr_hl_abs_cancel_m_as,CIR_large_order_weighted_high_depth_abs_5mean_order_m_bid,amount_deal_kendal_9corr_ov_order_day_ao,date,symbol
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.330433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-05,000001
1,0.223520,NaN,-1.000000,-1.000000,NaN,-1.000000,-1.000000,-1.000000,1.000000,NaN,...,-0.375499,0.044674,NaN,-4.688997,NaN,-1.000000,-0.022021,NaN,2015-01-06,000001
2,0.150423,0.084814,0.883178,-0.473739,1.000000,-0.988006,-0.919144,-1.000000,0.874829,0.013389,...,-0.369178,0.047949,-2.336827,-3.958000,-3.151480,-0.691711,-0.025666,NaN,2015-01-07,000001
3,0.146963,0.068207,0.904358,-0.133820,-0.409297,-0.978154,0.450277,-0.709803,0.715082,0.041117,...,-0.354230,0.044399,-2.367846,-4.387398,-2.900141,-0.390113,-0.043703,0.222222,2015-01-08,000001
4,0.139683,0.473910,0.873209,-0.349226,-0.409906,-0.982782,0.026403,-0.513922,0.661314,0.035876,...,-0.345370,0.042111,-2.581671,-3.870389,-3.242416,-0.619347,-0.047240,-0.212121,2015-01-09,000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10626485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-26,000562
10626486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-27,000562
10626487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-28,000562
10626488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-06-29,000562


In [73]:
dataframe.to_parquet('/root/gru/datasets/outstand.parq', index=False)

In [70]:
dataframe = pd.read_parquet('/root/gru/datasets/outstand.parq')

In [42]:
dates_with_nans = dataframe.drop(columns='symbol').isna().any(axis=1).groupby(dataframe['date']).any()
# 输出含有NaN值的日期
print(dates_with_nans[dates_with_nans].index.tolist())

[Timestamp('2015-01-05 00:00:00'), Timestamp('2015-01-06 00:00:00'), Timestamp('2015-01-07 00:00:00'), Timestamp('2015-01-08 00:00:00')]


In [41]:
cols_to_fill = dataframe.columns[dataframe.columns != 'symbol']
dataframe[cols_to_fill] = dataframe.groupby('date')[cols_to_fill].transform(lambda x: x.fillna(x.mean()))

In [8]:
dataset1[key].tensors[0].shape[0]

3820

In [9]:
dataset2[key].tensors[0].shape[0]

3835

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.2.0+cu121
True


In [96]:
file_path = '/root/backtest/factors/cross_Truenorm_c_30_input_size_6_prediction_size_10_window_30_hidden_dim_64_num_layers_3_MSE/1000/'
file_path+'data.parq'

'/root/backtest/factors/cross_Truenorm_c_30_input_size_6_prediction_size_10_window_30_hidden_dim_64_num_layers_3_MSE/1000/data.parq'

In [97]:
file_path = '/root/backtest/factors/cross_Truenorm_c_30_input_size_6_prediction_size_25_window_50_hidden_dim_64_num_layers_3_MSE_1d/1000/data.parq'
dataframe = pd.read_parquet(file_path)

In [16]:
begin_date = '2015-01-03'
end_date = '2016-12-31'
status = fds.list_status(begin_date, end_date, columns=None)
bars = fds.bar(begin_date,end_date, freq='1d')

In [17]:
bars

,date,symbol,pre_close,open,high,low,close,volume,turnover,match_items
0,2015-01-05,600000,15.69,15.88,16.25,15.56,16.07,513568709,8.182821e+09,126351
1,2015-01-05,600004,10.93,11.00,11.75,10.94,11.65,39217234,4.474540e+08,18347
2,2015-01-05,600005,3.58,3.58,3.82,3.57,3.77,258097632,9.583091e+08,47769
3,2015-01-05,600006,5.95,5.96,6.35,5.88,6.24,65370971,3.996788e+08,22849
4,2015-01-05,600007,15.29,15.28,16.28,15.20,15.76,10905429,1.728055e+08,6433
...,...,...,...,...,...,...,...,...,...,...
3029,2016-12-30,300577,49.95,54.95,54.95,54.95,54.95,15900,8.737050e+05,46
3030,2016-12-30,300579,28.08,30.89,30.89,30.89,30.89,3300,1.019370e+05,8
3031,2016-12-30,300581,67.99,74.79,74.79,74.79,74.79,20132,1.505672e+06,60
3032,2016-12-30,300582,23.52,25.87,25.87,25.87,25.87,5659,1.463983e+05,11


In [17]:
df = dataframe[dataframe['date']>pd.Timestamp('2021-01-03')]

In [31]:
columns_to_exclude = {'date', 'symbol'}
columns_to_fill = [col for col in df.columns if col not in columns_to_exclude]

df[columns_to_fill] = df[columns_to_fill].fillna(df.groupby('date')[columns_to_fill].transform('mean'))

In [33]:
df = df.groupby('symbol').apply(lambda group: group.ffill())

In [47]:
begin_date = '2014-01-03'
end_date = '2023-12-31'
status = fds.list_status(begin_date, end_date, columns=None)
bars = fds.bar(begin_date,end_date, freq='1d')
# 涨跌停筛选 截面日超过一定时间比例则删 flag为1 
threshold_of_grow_and_decline = 0.5
flager = fds.bar(begin_date,end_date, freq='5m')
high_max = flager.groupby(['date', 'symbol'])['high'].transform('max')
low_min = flager.groupby(['date', 'symbol'])['low'].transform('min')
flager['close_ge_high'] = (flager['close'] >= high_max).astype(int )
flager['close_le_low'] = (flager['close'] <= low_min).astype(int)
flags_high_ratio = flager.groupby(['date', 'symbol'])['close_ge_high'].transform('mean')
flags_low_ratio = flager.groupby(['date', 'symbol'])['close_le_low'].transform('mean')
flager['flag'] = ((flags_high_ratio > threshold_of_grow_and_decline) | (flags_low_ratio > threshold_of_grow_and_decline)).astype(int)
daily_flags = (flager.groupby(['date', 'symbol'])['flag']
               .first()
               .reset_index())
pools = {}
# PT当日无分钟频数据（PT）删除 上市日期小于等于10 每日流通量最低的5%  
# 每日成交比最少的4%股票PT大概在4.5% 最终删除率在10%左右
merged = pd.merge(bars, daily_flags, on=['date', 'symbol'], how='left')
merged = merged.reset_index(drop=True)
merged['ret'] = merged['close'] / (merged['pre_close'] + 1e-8) -1
merged['flag'] = merged['flag'].fillna(1)
merged = pd.merge(merged, status, on=['date', 'symbol'], how='left')
merged = merged.reset_index(drop=True)
merged[['ST','PT']] = merged[['ST','PT']].fillna(0)
merged['listed_days'] = merged['listed_days'].fillna(1)
merged['flag'] = merged['flag'].astype(int) | (merged['listed_days'] <= 10).astype(int)
merged['rank'] = merged.groupby('date')['turnover'].rank(pct=True)
merged.loc[merged['rank'] <= 0.05, 'flag'] = 1
merged.loc[merged['ret'].abs() > 0.25, 'flag'] = 1
# 股票池
filtered = merged[(merged['flag'] != 1)]
pools = {date: symbols.reset_index(drop=True) for date, symbols in filtered.groupby('date')['symbol']}
pools_df = pd.concat(pools.values(), keys=pools.keys()).reset_index().drop('level_1', axis=1)
pools_df.columns = ['date', 'symbol']

In [44]:
threshold_of_grow_and_decline = 0.5
flager = fds.bar('2020-02-03','2020-02-03', freq='5m')
high_max = flager.groupby(['date', 'symbol'])['high'].transform('max')
low_min = flager.groupby(['date', 'symbol'])['low'].transform('min')
flager['close_ge_high'] = (flager['close'] >= high_max).astype(int )
flager['close_le_low'] = (flager['close'] <= low_min).astype(int)
flags_high_ratio = flager.groupby(['date', 'symbol'])['close_ge_high'].transform('mean')
flags_low_ratio = flager.groupby(['date', 'symbol'])['close_le_low'].transform('mean')
flager['flag'] = ((flags_high_ratio > threshold_of_grow_and_decline) | (flags_low_ratio > threshold_of_grow_and_decline)).astype(int)

In [45]:
flager

,date,time,symbol,pre_close,open,high,low,close,volume,turnover,match_items,close_ge_high,close_le_low,flag
0,2020-02-03,0 days 09:30:00,000001,15.54,13.99,13.99,13.99,13.99,9644694,1.349293e+08,3005,0,1,0
1,2020-02-03,0 days 09:35:00,000001,13.99,14.00,14.40,13.99,14.24,20217107,2.866777e+08,8912,0,0,0
2,2020-02-03,0 days 09:40:00,000001,14.24,14.23,14.28,14.08,14.11,9850402,1.396566e+08,5365,0,0,0
3,2020-02-03,0 days 09:45:00,000001,14.11,14.11,14.37,14.11,14.36,8876736,1.262216e+08,4370,0,0,0
4,2020-02-03,0 days 09:50:00,000001,14.36,14.36,14.70,14.28,14.68,13025143,1.879146e+08,5241,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184235,2020-02-03,0 days 14:40:00,688399,117.48,117.48,117.48,117.48,117.48,1400,1.644720e+05,3,1,0,0
184236,2020-02-03,0 days 14:45:00,688399,117.48,117.48,117.48,117.48,117.48,7139,8.386897e+05,9,1,0,0
184237,2020-02-03,0 days 14:50:00,688399,117.48,117.48,117.48,117.48,117.48,8839,1.038406e+06,11,1,0,0
184238,2020-02-03,0 days 14:55:00,688399,117.48,117.48,117.48,117.48,117.48,7485,8.793378e+05,10,1,0,0


In [70]:
fold = 1
train_begin = pd.Timestamp(train_begin_date)
valid_end = pd.Timestamp(valid_end_date)
total_days = (valid_end - train_begin).days
days_per_fold = total_days // fold
validation_dates = []
for i in range(fold):
    validation_start = train_begin + pd.Timedelta(days=i*days_per_fold)
    validation_end = validation_start + pd.Timedelta(days=days_per_fold)
    validation_dates.append((validation_start, validation_end))

In [51]:
with open('datasets/pools.pkl', 'wb') as file:
    pickle.dump(pools, file)

In [74]:
with open('./datasets/pools.pkl', 'rb') as f:
        pools = pickle.load(f)

In [41]:
pools[pd.Timestamp('2020-02-03')]

0      600000
1      600004
2      600009
3      600015
4      600016
        ...  
625    300760
626    300766
627    300770
628    300771
629    300801
Name: symbol, Length: 630, dtype: object

In [17]:
#填补分钟频数据
begin_date = '2015-01-03'
end_date = '2015-03-03'
freq='60m'
prediction_size = 25
window = 50
ex = False
bars1 = fds.bar(fds.shift_trading_day(begin_date, -window-1),fds.shift_trading_day(end_date, prediction_size + 1), freq='1d')
bars2 = fds.bar(fds.shift_trading_day(begin_date, -window-1),fds.shift_trading_day(end_date, prediction_size + 1), freq='60m')
time_stamps = bars2['time'].unique()
time_df = pd.DataFrame({'time': time_stamps})
repeat_times = len(time_stamps)
dfnew = pd.DataFrame(np.repeat(bars1.values,repeat_times,axis=0))
dfnew.columns = bars1.columns
group_size = len(dfnew) // len(time_df)
time_df_expanded = pd.concat([time_df]*group_size, ignore_index=True)
dfnew = pd.concat([dfnew, time_df_expanded], axis=1)
dfnew = dfnew.sort_values(by = ['date','symbol']).reset_index(drop=True)
merged_df = pd.merge(dfnew, bars2, on=['date', 'symbol','time'], how='left', suffixes=('_df1', '_df2'))
columns_to_update = bars2.columns.difference(['date', 'symbol','time'])
for col in columns_to_update:
    merged_df[col] = merged_df[f'{col}_df2'].fillna(merged_df[f'{col}_df1'])
merged_df = merged_df[['date','symbol','time'] + columns_to_update.tolist()]

df = merged_df.copy()
adjusting_factor = fds.adjusting_factor(fds.shift_trading_day(begin_date, -window-1), fds.shift_trading_day(end_date, prediction_size + 1))
mg = pd.merge(df, adjusting_factor ,how= 'outer')
mg['ratio_adjusting_factor'].fillna(1, inplace=True)
mg['close'] = mg['close'] * mg['ratio_adjusting_factor']
mg['high'] = mg['high'] * mg['ratio_adjusting_factor']
mg['low'] = mg['low'] * mg['ratio_adjusting_factor']
mg['open'] = mg['open'] * mg['ratio_adjusting_factor']
mg['volume'] = mg['volume'] / (mg['ratio_adjusting_factor']+1e-8)
if ex:
    mg = mg.drop(['adjusting_factor','adjusting_const','ratio_adjusting_factor','pred_close','trade_vol','trade_val','preclose'], axis=1)
else:
    mg = mg.drop(['adjusting_factor','adjusting_const','ratio_adjusting_factor','pre_close','match_items'], axis=1)
mg = mg.dropna()
if freq == '1d':
    date_data = 1
else:
    date_data = mg.groupby(['date', 'symbol'])['time'].nunique().max()
mg['open_shift1'] = mg.groupby('symbol')['open'].shift(-1*date_data)
mg['open_shiftn'] = mg.groupby('symbol')['open'].shift(-(prediction_size + 1)*date_data)
mg['ret_predict'] = mg['open_shiftn']/(mg['open_shift1']+1e-8) -1
mg = mg.drop(['open_shiftn','open_shift1'], axis=1)
mg = mg.dropna()
df = mg.copy()
df = df.dropna()    
#ret_predict放到最后一列
ret_column = df.pop('ret_predict')
df['ret_predict'] = ret_column
df = df.reset_index(drop=True)

In [276]:
unique_dates = df['date'].unique()
shifted_dates = {date: fds.shift_trading_day(date, -window) for date in unique_dates}
dic = {}
dataset = {}
for date in df['date'].unique():
    if freq == '1d':
        tensors_dict = process_date(df=df,pools=pools,window=window,shifted_dates=shifted_dates,date=date) 
    else:
        tensors_dict = process_date_minutes(df=df,pools=pools,window=window,shifted_dates=shifted_dates,date=date,date_data=date_data)  
    if len(tensors_dict) != 0  :
        dic[date] = tensors_dict

In [277]:
df_list = [pd.DataFrame({'date': outer_key, 'symbol': list(inner_dict.keys())})
           for outer_key, inner_dict in dic.items()]
df_keys = pd.concat(df_list, ignore_index=True)
norm = True
for key in dic.keys():
    day_tensor = torch.stack([tensor for tensor in dic[key].values()],dim=0)
    y = day_tensor[:,-date_data,-1]
    X = day_tensor[:,:,0:-1]
    if norm:
        # X时序标准化
        X = X / (X[: , -1 , :].unsqueeze(1)+1e-8)
        # X = (X - mean_seq) / (std_seq + 1e-8)
        # X截面标准化
        mean_cross = X.mean(dim=0, keepdim=True)  
        std_cross = X.std(dim=0, keepdim=True)    
        X = (X - mean_cross) / (std_cross + 1e-8) 
        y  = (y-y.mean())/(y.std()+1e-8) 
    X.requires_grad_(True)
    y.requires_grad_(True)
    dataset[key] = TensorDataset(X,y)

In [278]:
X = dataset[pd.Timestamp('2015-01-05')].tensors[0]
y = dataset[pd.Timestamp('2015-01-05')].tensors[1]

In [279]:
X.shape

torch.Size([2265, 250, 6])

In [167]:
y

tensor([-0.0113, -0.0179, -0.0552,  ..., -0.0282, -0.0593, -0.0179],
       requires_grad=True)

In [168]:
a = df[df['date'] == pd.Timestamp('2019-05-28')]

In [175]:
a[a['time']=='0 days 09:30:00']

,date,symbol,time,close,high,low,open,turnover,volume,ret_predict
2241770,2019-05-28,000001,0 days 09:30:00,1758.801130,1758.801130,1758.801130,1758.801130,4394670.0,2498.673628,-0.011327
2241775,2019-05-28,000002,0 days 09:30:00,3080.196764,3080.196764,3080.196764,3080.196764,3053700.0,991.397704,-0.017949
2241780,2019-05-28,000004,0 days 09:30:00,135.413878,135.413878,135.413878,135.413878,869400.0,6420.316824,-0.055163
2241785,2019-05-28,000005,0 days 09:30:00,30.287498,30.287498,30.287498,30.287498,123200.0,4067.684992,-0.029126
2241790,2019-05-28,000006,0 days 09:30:00,356.333685,356.333685,356.333685,356.333685,98212.0,275.618063,-0.019643
...,...,...,...,...,...,...,...,...,...,...
2259850,2019-05-28,603993,0 days 09:30:00,12.860073,12.860073,12.860073,12.860073,629586.0,48956.642973,0.027990
2259855,2019-05-28,603996,0 days 09:30:00,9.659761,9.659761,9.659761,9.659761,0.0,0.000000,-0.142140
2259860,2019-05-28,603997,0 days 09:30:00,12.819732,12.819732,12.819732,12.819732,88144.0,6875.650615,-0.028241
2259865,2019-05-28,603998,0 days 09:30:00,36.623825,36.623825,36.623825,36.623825,2702693.0,73796.032417,-0.059341


In [2]:
begin_date = '2018-01-03'
end_date = '2018-03-31'
adds = True
prediction_size = 3
window = 30
# bars = fds.bar(fds.shift_trading_day(begin_date, -window-1), fds.shift_trading_day(end_date, prediction_size + 1), freq='1d')
bars = fds.exbar_mini(fds.shift_trading_day(begin_date, -window-1), fds.shift_trading_day(end_date, prediction_size + 1), freq='1d')
bars = bars[['date','symbol','open','close','high','low','volume','turnover','trade_num', #高开低收量额 ，买卖， 大小单
             'init_buy_trade_num','init_buy_trade_vol','init_buy_trade_val','init_sell_trade_num','init_sell_trade_vol','init_sell_trade_val',
             'large_trade_num','large_trade_vol','large_trade_val','med_trade_num','med_trade_vol','med_trade_val','small_trade_num','small_trade_vol','small_trade_val']]
bars=ratio_bars(bars)
bars = remove_outliers_mad(bars)
# bars = fds.exbar_mini(fds.shift_trading_day(begin_date, -window+1), end_date, freq='1d')
adjusting_factor = fds.adjusting_factor(fds.shift_trading_day(begin_date, -window-1), fds.shift_trading_day(end_date, prediction_size + 1))
mg = pd.merge(bars, adjusting_factor ,how= 'outer')
mg['ratio_adjusting_factor'].fillna(1, inplace=True)
mg['close'] = mg['close'] * mg['ratio_adjusting_factor']
mg['high'] = mg['high'] * mg['ratio_adjusting_factor']
mg['low'] = mg['low'] * mg['ratio_adjusting_factor']
mg['open'] = mg['open'] * mg['ratio_adjusting_factor']
mg['volume'] = mg['volume'] / (mg['ratio_adjusting_factor']+1e-8)
mg = mg.drop(['adjusting_factor','adjusting_const','ratio_adjusting_factor'], axis=1)
# mg = mg.drop(['adjusting_factor','adjusting_const','ratio_adjusting_factor','pre_close','match_items'], axis=1)
# mg = mg.drop(['adjusting_factor','adjusting_const','ratio_adjusting_factor','pred_close','trade_vol','trade_val','preclose'], axis=1)

mg['open_shiftn'] = mg.groupby('symbol')['open'].shift(-prediction_size-1)
mg['open_shift1'] = mg.groupby('symbol')['open'].shift(-1)
mg['ret_predict'] = mg['open_shiftn']/(mg['open_shift1']+1e-8) -1
mg = mg.drop(['open_shift1','open_shiftn'], axis=1)
mg = mg.dropna()
bars = mg.copy()

In [6]:
bars.columns

Index(['date', 'symbol', 'open', 'close', 'high', 'low', 'volume', 'turnover',
       'trade_num', 'init_buy_trade_num', 'init_buy_trade_vol',
       'init_buy_trade_val', 'init_sell_trade_num', 'init_sell_trade_vol',
       'init_sell_trade_val', 'large_trade_num', 'large_trade_vol',
       'large_trade_val', 'med_trade_num', 'med_trade_vol', 'med_trade_val',
       'small_trade_num', 'small_trade_vol', 'small_trade_val', 'ret_predict'],
      dtype='object')

In [53]:
df = df.dropna()    
#ret_predict放到最后一列
ret_column = df.pop('ret_predict')
df['ret_predict'] = ret_column
df = df.reset_index(drop=True)

In [255]:
def process_date(df, date):
    if date not in pools:
        return None

    start_time = time.time()
    valid_symbols = pools[date]
    mask = (df['symbol'].isin(valid_symbols)) & (df['date'] <= date) & (df['date'] > shifted_dates[date])
    data = df[mask]
    data = data.sort_values(by = ['symbol','date']).reset_index(drop=True)
    data = data.reset_index(drop=True)
    all_data_tensor = torch.tensor(data.drop(['date', 'symbol'], axis=1).values, dtype=torch.float)
    symbols = data['symbol'].values
    unique_symbols, counts = np.unique(symbols, return_counts=True)
    tensors = dict(zip(unique_symbols, torch.split(all_data_tensor, counts.tolist())))
    tensors_dict = {symbol: tensor for symbol, tensor in tensors.items() if tensor.shape[0] >= window*1}
    elapsed_time = time.time() - start_time
    print(f"Date {date} processed in {elapsed_time:.2f} seconds")
    return tensors_dict 
dic = {}
window = 30
unique_dates = bars['date'].unique()
shifted_dates = {date: fds.shift_trading_day(date, -window) for date in unique_dates}

# with Pool(processes=16) as pool:  
#     results = pool.map(process_date, unique_dates)
for date in unique_dates:
    tensors_dict = process_date(bars, date) 
    if len(tensors_dict) != 0 :
        dic[date] = tensors_dict
# dic = {date: result for date, result in zip_longest(unique_dates, results) if result is not None}

Date 2014-11-19 00:00:00 processed in 0.21 seconds
Date 2014-11-20 00:00:00 processed in 0.22 seconds
Date 2014-11-21 00:00:00 processed in 0.23 seconds
Date 2014-11-24 00:00:00 processed in 0.22 seconds
Date 2014-11-25 00:00:00 processed in 0.21 seconds
Date 2014-11-26 00:00:00 processed in 0.26 seconds
Date 2014-11-27 00:00:00 processed in 0.24 seconds
Date 2014-11-28 00:00:00 processed in 0.24 seconds
Date 2014-12-01 00:00:00 processed in 0.21 seconds
Date 2014-12-02 00:00:00 processed in 0.23 seconds
Date 2014-12-03 00:00:00 processed in 0.25 seconds
Date 2014-12-04 00:00:00 processed in 0.22 seconds
Date 2014-12-05 00:00:00 processed in 0.23 seconds
Date 2014-12-08 00:00:00 processed in 0.25 seconds
Date 2014-12-09 00:00:00 processed in 0.25 seconds
Date 2014-12-10 00:00:00 processed in 0.23 seconds
Date 2014-12-11 00:00:00 processed in 0.22 seconds
Date 2014-12-12 00:00:00 processed in 0.24 seconds
Date 2014-12-15 00:00:00 processed in 0.23 seconds
Date 2014-12-16 00:00:00 proces

In [70]:
date = pd.Timestamp('2020-03-05')
valid_symbols = pools[date]
mask = (df['symbol'].isin(valid_symbols)) & (df['date'] <= date) & (df['date'] > shifted_dates[date])
date_data = df[mask]
date_data = date_data.sort_values(by = ['symbol','date'])
date_data = date_data.reset_index(drop=True)
all_data_tensor = torch.tensor(date_data.drop(['date', 'symbol'], axis=1).values, dtype=torch.float)
# symbols = date_data['symbol'].values
# unique_symbols, counts = np.unique(symbols, return_counts=True)
# tensors = dict(zip(unique_symbols, torch.split(all_data_tensor, counts.tolist())))
# tensors_dict = {symbol: tensor for symbol, tensor in tensors.items() if tensor.shape[0] >= window}
# elapsed_time = time.time() - start_time
# print(f"Date {date} processed in {elapsed_time:.2f} seconds")

In [16]:
for key1, sub_dic in dic.items():
    for key2, tensor in sub_dic.items():
        if torch.isnan(tensor).any():
            print(f"在字典中找到了NaN值，对应的键是'{key1}' -> '{key2}'")

In [256]:
dict_dataset= {}
for key in dic.keys():
    day_tensor = torch.stack([tensor for tensor in dic[key].values()],dim=0)
    y = day_tensor[:,-1,-1]
    X = day_tensor[:,:,0:-1]
    
    # X时序标准化
    # mean_seq = X.mean(dim=1, keepdim=True)
    # std_seq = X.std(dim=1, keepdim=True)
    # X = (X - mean_seq) / (std_seq + 1e-8)
    # # X截面标准化
    # mean_cross = X.mean(dim=0, keepdim=True)  
    # std_cross = X.std(dim=0, keepdim=True)    
    # X = (X - mean_cross) / (std_cross + 1e-8)  

    #y截面标准化
    # y  = (y-y.mean())/(y.std()+1e-8)
    dict_dataset[key] = TensorDataset(X,y)

In [184]:
X = dict_dataset[pd.Timestamp('2019-05-28')].tensors[0]
y = dict_dataset[pd.Timestamp('2019-05-28')].tensors[1]

In [151]:
X

tensor([[[1.9274e+03, 1.9417e+03, 1.8874e+03, 1.9174e+03, 5.4196e+05,
          1.0351e+09],
         [1.9603e+03, 2.0131e+03, 1.9517e+03, 1.9560e+03, 1.0736e+06,
          2.1347e+09],
         [1.9531e+03, 2.0831e+03, 1.9360e+03, 2.0831e+03, 1.6653e+06,
          3.3796e+09],
         ...,
         [1.7645e+03, 1.7788e+03, 1.7588e+03, 1.7645e+03, 3.4682e+05,
          6.1311e+08],
         [1.7445e+03, 1.7745e+03, 1.7045e+03, 1.7674e+03, 7.3380e+05,
          1.2756e+09],
         [1.7588e+03, 1.7931e+03, 1.7517e+03, 1.7845e+03, 6.1641e+05,
          1.0941e+09]],

        [[3.6015e+03, 3.6244e+03, 3.5148e+03, 3.5468e+03, 4.2289e+05,
          1.5045e+09],
         [3.6244e+03, 3.6506e+03, 3.5228e+03, 3.5297e+03, 4.8896e+05,
          1.7593e+09],
         [3.5080e+03, 3.6141e+03, 3.4236e+03, 3.6095e+03, 6.2165e+05,
          2.1856e+09],
         ...,
         [3.0688e+03, 3.0859e+03, 3.0357e+03, 3.0597e+03, 1.7341e+05,
          5.3046e+08],
         [3.0562e+03, 3.0893e+03, 3.0003

In [185]:
y

tensor([-0.0113, -0.0179, -0.0552,  ..., -0.0282, -0.0593, -0.0179])

In [187]:
dict_dataset[pd.Timestamp('2019-03-05')].tensors[0].shape

torch.Size([3298, 30, 6])

In [188]:
dataset[pd.Timestamp('2019-03-05')].tensors[0].shape

torch.Size([3298, 150, 6])

In [250]:
for key, dicc in dict_dataset.items():
    try:
        assert dict_dataset[key].tensors[0].shape[0] == dataset[key].tensors[0].shape[0]
    except AssertionError:
        print(f"Assertion failed for key: {key}")

In [24]:
for key, tensor in dict_dataset.items():
    if torch.isnan(tensor.tensors[0]).any() | torch.isnan(tensor.tensors[1]).any():
        print(f"在字典中找到了NaN值，对应的键是'{key}'")

In [34]:
df_list = [pd.DataFrame({'date': outer_key, 'symbol': list(inner_dict.keys())})
           for outer_key, inner_dict in dic.items()]
df_keys = pd.concat(df_list, ignore_index=True)

In [47]:
df_keys = df_keys.sort_values(by = ['date','symbol'])
df_keys.reset_index(drop=True)

,date,symbol
0,2019-07-09,000001
1,2019-07-09,000002
2,2019-07-09,000004
3,2019-07-09,000005
4,2019-07-09,000006
...,...,...
659335,2020-06-23,688388
659336,2020-06-23,688389
659337,2020-06-23,688396
659338,2020-06-23,688398


In [60]:
from model import *
from utilities import *
model_name = './models/nonorm_train_begin_date_2015-01-03_c_30_input_size_20_prediction_size_10_window_30_hidden_dim_256_num_layers_3_Cos_train.pth'
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
input_size = 20
hidden_dim = 256
num_layers  = 3
output_dim = 1
c = 30
num_heads = 2
dropout_rate = 0.05
model = CrossGRU(input_size, hidden_dim, num_layers, output_dim, dropout_rate, c, num_heads, device).to(device)
checkpoint = torch.load(model_name, map_location = device)
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [65]:
model.eval()
criterion = CosineLoss().to(device)
with torch.no_grad():
    outputs = []
    losses = []
    keys = list(dict_dataset.keys())
    for key in keys:
        data, target = dict_dataset[key].tensors[0] , dict_dataset[key].tensors[1]
        data, target = data.to(device), target.to(device)
        output = model(data).view(-1)
        target = target.view(-1)
        loss = criterion(output, target)
        outputs.append(output)
        losses.append(loss)

In [87]:
backtest = None
if backtest is None:
    backtest = torch.cat(outputs).cpu().numpy()
else:
    backtest += torch.cat(outputs).cpu().numpy()

In [89]:
df_keys['nonorm_train_begin_date_2015-01-03_c_30_input_size_20_prediction_size_10_window_30_hidden_dim_256_num_layers_3_Cos'] = backtest

In [90]:
df_keys

,date,symbol,nonorm_train_begin_date_2015-01-03_c_30_input_size_20_prediction_size_10_window_30_hidden_dim_256_num_layers_3_Cos
0,2019-07-09,000001,34.334461
1,2019-07-09,000002,34.334019
2,2019-07-09,000004,34.351387
3,2019-07-09,000005,34.366436
4,2019-07-09,000006,34.346905
...,...,...,...
659335,2020-06-23,688388,34.335365
659336,2020-06-23,688389,34.342167
659337,2020-06-23,688396,34.335922
659338,2020-06-23,688398,34.335247


In [154]:
df_keys.to_parquet('factor/{}.parq'.format('input_size_24_prediction_size_10_window_30_hidden_dim_512_num_layers_3_epoch_2'))

In [143]:
def generate_pools(train_begin_date : Union[pd.Timestamp, str],test_end_date : Union[pd.Timestamp, str],
                    threshold_of_grow_and_decline = 0.5):
    """
    返回一个字典，一个dataframe
    """
    status = fds.list_status(train_begin_date, test_end_date, columns=None)
    bars = fds.bar(train_begin_date, test_end_date, freq='1d')
    # 涨跌停筛选 截面日超过一定时间比例则删 flag为1 
    flager = fds.bar(train_begin_date, test_end_date, freq='5m')
    high_max = flager.groupby(['date', 'symbol'])['high'].transform('max')
    low_min = flager.groupby(['date', 'symbol'])['low'].transform('min')
    flager['close_ge_high'] = (flager['close'] >= high_max).astype(int )
    flager['close_le_low'] = (flager['close'] <= low_min).astype(int)
    flags_high_ratio = flager.groupby(['date', 'symbol'])['close_ge_high'].transform('mean')
    flags_low_ratio = flager.groupby(['date', 'symbol'])['close_le_low'].transform('mean')
    flager['flag'] = ((flags_high_ratio > threshold_of_grow_and_decline) | (flags_low_ratio > threshold_of_grow_and_decline)).astype(int)
    daily_flags = (flager.groupby(['date', 'symbol'])['flag']
                .first()
                .reset_index())
    pools = {}
    # PT当日无分钟频数据（PT）删除 上市日期小于等于10 每日流通量最低的5%  
    # 每日成交比最少的4%股票PT大概在4.5% 最终删除率在10%左右
    merged = pd.merge(bars, daily_flags, on=['date', 'symbol'], how='left')
    merged['flag'] = merged['flag'].fillna(1)
    merged = pd.merge(merged, status, on=['date', 'symbol'], how='left')
    merged[['ST','PT']] = merged[['ST','PT']].fillna(0)
    merged['listed_days'] = merged['listed_days'].fillna(1)
    merged['flag'] = merged['flag'].astype(int) | (merged['listed_days'] <= 10).astype(int)
    merged['rank'] = merged.groupby('date')['match_items'].rank(pct=True)
    merged.loc[merged['rank'] <= 0.05, 'flag'] = 1

    # 股票池
    filtered = merged[(merged['flag'] != 1)]
    pools = {date: symbols.reset_index(drop=True) for date, symbols in filtered.groupby('date')['symbol']}
    pools_df = pd.concat(pools.values(), keys=pools.keys()).reset_index().drop('level_1', axis=1)
    pools_df.columns = ['date', 'symbol']
    return pools , pools_df

def time_series_dataset_dict(train_begin_date : Union[pd.Timestamp, str], train_end_date : Union[pd.Timestamp, str], 
                            valid_begin_date : Union[pd.Timestamp, str], valid_end_date : Union[pd.Timestamp, str],
                            test_begin_date : Union[pd.Timestamp, str], test_end_date : Union[pd.Timestamp, str],
                            threshold_of_grow_and_decline = 0.5, prediction_size = 10,
                            window = 30):
    bars = fds.bar(train_begin_date, test_end_date, freq='1d')
    pools , pools_df = generate_pools(train_begin_date , test_end_date, threshold_of_grow_and_decline)

    df = bars.copy()
    train_df = df[df['date'] <= pd.Timestamp(train_end_date)]
    valid_df = df[(df['date'] >= pd.Timestamp(fds.shift_trading_day(valid_begin_date, -window+1))) & (df['date'] <= pd.Timestamp(valid_end_date))]
    test_df = df[df['date'] >= pd.Timestamp(fds.shift_trading_day(test_begin_date, -window+1))]
    unique_dates = df['date'].unique()
    shifted_dates = {date: fds.shift_trading_day(date, -window) for date in unique_dates}

    def ret_df(df,prediction_size):
        df['ret'] = df['close']/df['pre_close']-1
        df['ret_predict'] = df.groupby('symbol')['ret'].transform(
            lambda x: x.rolling(window=prediction_size).apply(lambda y: (y + 1).prod(), raw=True) - 1
        )
        df['ret_predict'] = df.groupby('symbol')['ret_predict'].shift(-prediction_size-1)
        return df.dropna()
    
    def dic_df(df,prediction_size):
        df = ret_df(df,prediction_size)
        dic = {}
        dataset = {}
        for date in df['date'].unique():
            tensors_dict = process_date(df=df,pools=pools,window=window,shifted_dates=shifted_dates,date=date) 
            if len(tensors_dict) != 0  :
                dic[date] = tensors_dict
        for key in dic.keys():
            day_tensor = torch.stack([tensor for tensor in dic[key].values()],dim=0)
            y = day_tensor[:,-1,-1]
            X = day_tensor[:,:,0:-1]
            mean = X.mean(dim=1, keepdim=True)
            std = X.std(dim=1, keepdim=True)
            X = (X - mean) / std
            dataset[key] = TensorDataset(X,y)
        return dataset
    
    train_dataset = dic_df(train_df,prediction_size)
    valid_dataset = dic_df(valid_df,prediction_size)
    test_dataset = dic_df(test_df,prediction_size)

    return train_dataset , valid_dataset , test_dataset
    

def process_date(df, pools, window, shifted_dates, date): 
    if date not in pools:
        return None
    valid_symbols = pools[date]
    mask = (df['symbol'].isin(valid_symbols)) & (df['date'] <= date) & (df['date'] > shifted_dates[date])
    date_data = df[mask]
    date_data = date_data.sort_values(by = ['symbol','date'])
    all_data_tensor = torch.tensor(date_data.drop(['date', 'symbol'], axis=1).values, dtype=torch.float)
    symbols = date_data['symbol'].values
    unique_symbols, counts = np.unique(symbols, return_counts=True)
    tensors = dict(zip(unique_symbols, torch.split(all_data_tensor, counts.tolist())))
    tensors_dict = {symbol: tensor for symbol, tensor in tensors.items() if tensor.shape[0] >= window}
    return tensors_dict 

In [ ]:
train_begin_date = '2015-01-03'
train_end_date = '2018-12-31'
valid_begin_date = '2019-01-03'
valid_end_date = '2019-12-31'
test_begin_date = '2020-01-03'
test_end_date = '2020-12-31'

train_dataset , valid_dataset , test_dataset = time_series_dataset_dict(
                    train_begin_date = '2015-01-03', train_end_date = '2018-12-31', 
                   valid_begin_date = '2019-01-03', valid_end_date = '2019-12-31',
                   test_begin_date = '2020-01-03',test_end_date = '2020-12-31',
                   threshold_of_grow_and_decline = 0.5, prediction_size = 10,
                   window = 30)

In [164]:
def generate_pools(train_begin_date : Union[pd.Timestamp, str],test_end_date : Union[pd.Timestamp, str],
                    threshold_of_grow_and_decline = 0.5 ):
    """
    返回一个字典，一个dataframe
    """
    status = fds.list_status(train_begin_date, test_end_date, columns=None)
    bars = fds.bar(train_begin_date, test_end_date, freq='1d')
    # 涨跌停筛选 截面日超过一定时间比例则删 flag为1 
    flager = fds.bar(train_begin_date, test_end_date, freq='5m')
    high_max = flager.groupby(['date', 'symbol'])['high'].transform('max')
    low_min = flager.groupby(['date', 'symbol'])['low'].transform('min')
    flager['close_ge_high'] = (flager['close'] >= high_max).astype(int )
    flager['close_le_low'] = (flager['close'] <= low_min).astype(int)
    flags_high_ratio = flager.groupby(['date', 'symbol'])['close_ge_high'].transform('mean')
    flags_low_ratio = flager.groupby(['date', 'symbol'])['close_le_low'].transform('mean')
    flager['flag'] = ((flags_high_ratio > threshold_of_grow_and_decline) | (flags_low_ratio > threshold_of_grow_and_decline)).astype(int)
    daily_flags = (flager.groupby(['date', 'symbol'])['flag']
                .first()
                .reset_index())
    pools = {}
    # PT当日无分钟频数据（PT）删除 上市日期小于等于10 每日流通量最低的5%  
    # 每日成交比最少的4%股票PT大概在4.5% 最终删除率在10%左右
    merged = pd.merge(bars, daily_flags, on=['date', 'symbol'], how='left')
    merged['flag'] = merged['flag'].fillna(1)
    merged = pd.merge(merged, status, on=['date', 'symbol'], how='left')
    merged[['ST','PT']] = merged[['ST','PT']].fillna(0)
    merged['listed_days'] = merged['listed_days'].fillna(1)
    merged['flag'] = merged['flag'].astype(int) | (merged['listed_days'] <= 10).astype(int)
    merged['rank'] = merged.groupby('date')['match_items'].rank(pct=True)
    merged.loc[merged['rank'] <= 0.05, 'flag'] = 1

    # 股票池
    filtered = merged[(merged['flag'] != 1)]
    pools = {date: symbols.reset_index(drop=True) for date, symbols in filtered.groupby('date')['symbol']}
    pools_df = pd.concat(pools.values(), keys=pools.keys()).reset_index().drop('level_1', axis=1)
    pools_df.columns = ['date', 'symbol']
    return pools , pools_df

In [165]:
train_begin_date = '2010-01-03'
test_end_date = '2023-12-31'
pools , pools_df = generate_pools(train_begin_date , test_end_date, threshold_of_grow_and_decline)

In [166]:
with open('./datasets/pools.pkl', 'wb') as f:
    pickle.dump(pools, f)
pools_df.to_pickle('./datasets/pools_df.pkl')

In [28]:
def process_date(df, pools, window, shifted_dates, date): 
    if date not in pools:
        return None
    valid_symbols = pools[date]
    mask = (df['symbol'].isin(valid_symbols)) & (df['date'] <= date) & (df['date'] > shifted_dates[date])
    date_data = df[mask]
    if len(date_data)<window:
        return {} 
    date_data = date_data.sort_values(by = ['symbol','date'])
    all_data_tensor = torch.tensor(date_data.drop(['date', 'symbol'], axis=1).values, dtype=torch.float)
    symbols = date_data['symbol'].values
    unique_symbols, counts = np.unique(symbols, return_counts=True)
    tensors = dict(zip(unique_symbols, torch.split(all_data_tensor, counts.tolist())))
    tensors_dict = {symbol: tensor for symbol, tensor in tensors.items() if tensor.shape[0] >= window}
    return tensors_dict 

In [29]:
test_begin_date = '2019-06-01'
test_end_date = '2020-06-30'
threshold_of_grow_and_decline = 0.5
window = 30
prediction_size = 10

with open('./datasets/pools.pkl', 'rb') as f:
    pools = pickle.load(f)

bars = fds.bar(fds.shift_trading_day(test_begin_date, -window+1), test_end_date, freq='1d')

unique_dates = bars['date'].unique()
shifted_dates = {date: fds.shift_trading_day(date, -window) for date in unique_dates}
bars['ret'] = bars['close']/bars['pre_close']-1
bars['ret_predict'] = bars.groupby('symbol')['ret'].transform(
    lambda x: x.rolling(window=prediction_size).apply(lambda y: (y + 1).prod(), raw=True) - 1
)
bars['ret_predict'] = bars.groupby('symbol')['ret_predict'].shift(-prediction_size-1)
bars =  bars.dropna()

dic = {}
dataset = {}
for date in bars['date'].unique():
    tensors_dict = process_date(df=bars,pools=pools,window=window,shifted_dates=shifted_dates,date=date) 
    if len(tensors_dict) != 0  :
        dic[date] = tensors_dict

df_list = [pd.DataFrame({'date': outer_key, 'symbol': list(inner_dict.keys())})
           for outer_key, inner_dict in dic.items()]
df_keys = pd.concat(df_list, ignore_index=True)

for key in dic.keys():
    day_tensor = torch.stack([tensor for tensor in dic[key].values()],dim=0)
    y = day_tensor[:,-1,-1]
    X = day_tensor[:,:,0:-1]
    mean = X.mean(dim=1, keepdim=True)
    std = X.std(dim=1, keepdim=True)
    X = (X - mean) / (std + 1e-8)
    dataset[key] = TensorDataset(X,y)

output_dim = 1
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [62]:
input_size =9
output_dim = 1
window = 30
hidden_dim = 512
num_layers = 3

model_name='/root/gru/models/input_size_9_prediction_size_10_window_30_hidden_dim_512_num_layers_3_train.pth'
factor_name = 'input_size_9_prediction_size_10_window_30_hidden_dim_512_num_layers_3'
device = 'cuda:0'
checkpoint = torch.load(model_name, map_location = device)
model = LSTM(input_size, hidden_dim, num_layers, output_dim).to(device)
checkpoint = torch.load(model_name, map_location = device)
model.load_state_dict(checkpoint['model'])
model.eval()
with torch.no_grad():
    outputs = []

    keys = list(dataset.keys())
    for key in keys:
        data, target = dataset[key].tensors[0] , dataset[key].tensors[1]
        data, target = data.to(device), target.to(device)
        target = target.view(-1)
        outputs.append(model(data))

In [68]:
df_keys[factor_name] = torch.cat(outputs).cpu().numpy()

In [69]:
df_keys

,date,symbol,input_size_9_prediction_size_10_window_30_hidden_dim_512_num_layers_3
0,2019-06-03,000001,-31.284882
1,2019-06-03,000002,-54.844955
2,2019-06-03,000004,-50.193790
3,2019-06-03,000005,-52.403088
4,2019-06-03,000006,-39.181267
...,...,...,...
871325,2020-06-11,688388,-49.414318
871326,2020-06-11,688389,-30.349098
871327,2020-06-11,688396,-8.636230
871328,2020-06-11,688398,-31.049786


In [14]:
input_size = 22
hidden_dim=64
num_layers=3
output_dim=1
dropout_rate=0.4
c=30
num_heads=2
device = 'cuda:1'
model = CrossGRUMLP(input_size, hidden_dim, num_layers, output_dim, dropout_rate, c, num_heads, device)

In [17]:
model.load_state_dict(torch.load('./models/crossmlp_Truerandom_sample_Trueex_dropout_0.4_train_begin_date_2020-01-03_c_30_input_size_22_prediction_size_15_window_30_hidden_dim_64_ratio_True_MSE_1d_1year_train.pth')['model'])

<All keys matched successfully>

In [18]:
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print(f"NaN found in parameter: {name}")

In [13]:
dataset1 , df_keys1  = time_series_dataset_dict_cross(
            begin_date = '2015-01-03',  end_date = '2017-03-03',
            threshold_of_grow_and_decline = 0.5, 
            prediction_size = 25, adds = False,
            window = 50, drop = False, norm = True , freq = '1d', ex = False, outstand = False)

In [23]:
for key, tensor in dataset1.items():
    if torch.isnan(dataset1[key].tensors[1]).any():
        print(f"NaN found in parameter: {key}")


In [26]:
dataset1[pd.Timestamp('2020-01-22')].tensors[0]

tensor([[[ 0.3234,  0.4235,  0.3732,  ..., -0.1345, -0.2856, -0.2846],
         [ 0.5217,  0.4275,  0.3979,  ..., -0.1500, -0.2814, -0.2806],
         [ 0.6290,  0.6406,  0.6265,  ...,  0.1083, -0.0760, -0.0754],
         ...,
         [ 0.5353,  0.2710,  0.4679,  ...,  0.3056,  0.1325,  0.1330],
         [ 0.3830, -0.2312,  0.0358,  ...,  0.9139,  0.6959,  0.6975],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.2514,  0.0278,  0.0224,  ..., -0.1309, -0.2619, -0.2618],
         [ 0.2891,  0.0543,  0.0239,  ..., -0.1463, -0.2575, -0.2576],
         [ 0.3263,  0.0041,  0.0403,  ...,  0.1121, -0.0506, -0.0511],
         ...,
         [ 0.8287, -0.6142, -0.0924,  ...,  0.3110,  0.1671,  0.1662],
         [-0.1210, -1.1948, -1.0606,  ...,  0.1870,  0.0556,  0.0546],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.5901, -0.7614, -0.7252,  ...,  1.5894,  1.5362,  1.5336],
         [-0.6844, -0.8078, -0.8246,  ...,  1

In [7]:
df = pd.read_parquet('factor/crossmlp_Truewinsor_Trueex_c_30_input_size_22_prediction_size_15_window_40_hidden_dim_64_ratio_True_IC_1d_1year.parq')

In [8]:
df

,date,symbol,crossmlp_Truewinsor_Trueex_c_30_input_size_22_prediction_size_15_window_40_hidden_dim_64_ratio_True_IC_1d_1year
744609,2021-04-01,000001,0.000000e+00
1489218,2021-04-01,000001,9.995191e-01
2233827,2021-04-01,000001,0.000000e+00
2978436,2021-04-01,000001,1.470012e-20
3723045,2021-04-01,000001,3.023457e-20
...,...,...,...
5956871,2021-12-31,689009,1.125603e-11
6701480,2021-12-31,689009,1.187881e-11
7446089,2021-12-31,689009,1.255784e-11
8190698,2021-12-31,689009,1.325060e-11
